# 第11回 その1: 中間層が1層のニューラルネットワークの実装
10_03_neural_network.ipynbでは中間層が1層でノード数が2のニューラルネットワークを実装しました。  
ここでは，中間層が1層で，ノード数が可変にできるようにしたソースコードを示します。

## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/11"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/11/
# 現在のフォルダの中身を表示
%ls

`2class_data_circle.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込みと標準化
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`2class_data_circle.csv` を読み込みます。  
このデータはx1とx2の二次元データで，クラス0とクラス1の2クラスのどちらかに属しています。  

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('2class_data_circle.csv', encoding='SHIFT-JIS')

# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# x1とx2のデータを抽出し，numpy用データ(ndarray型) に変換する。
X = csv_data.loc[:, ['x1','x2']].to_numpy()

# クラス番号を抽出し，numpy用データ(ndarray型) に変換する。
Y = csv_data.loc[:,'class'].to_numpy()

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Nunber of samples: ' + str(num_samples))
print('Number of dimensions: ' + str(num_dimensions))

# クラス数を得る。
num_classes = int(np.max(Y) + 1)
print('Number of classes: ' + str(num_classes))

# データを標準化する。
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

color_list = [(0, 0, 1), (1, 0, 0)]
# データをプロット
plt.figure(figsize=(7,7))
for k in range(num_classes):
  # n番目のクラスに属している(Y==c)データをプロット
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

いま，yには各データの正解クラス番号(0 or 1)が格納されていますが，  
クロスエントロピー損失の計算や勾配の計算には，クラス番号ではなく，各クラスの正解の確率が必要です。  
そのため，正解クラスは確率=1，不正解クラスは確率=0となるベクトルに変換しておきます。これを<font color=red>**one-hotベクトル**</font>と呼びます。

In [ ]:
# ゼロ行列を生成
Y_onehot = np.zeros([num_samples, num_classes])
for n in range(num_samples):
  # クラス番号(y[n])に対応するインデクスを1にする
  Y_onehot[n, Y[n]] = 1

# 先頭の5サンプル分を表示
print('class Y:')
print(Y[:5])
print('one-hot vector Y_onehot:')
print(Y_onehot[:5])

Yが[0, 1, 1, 0, 1]なのに対して，Y_onehotの各行は0, 1, 1, 0, 1番目の要素が1に，それ以外が0になっているのが分かります。

## ステップ2: 中間層のノード数が可変の3層ニューラルネットワークの実装  
10_03_neural_network.ipynbでは，中間層のノード数が2と限定した実装になっていましたが，  
ここではノード数を可変にできるような実装にします。  

入力の次元数を$D$，クラス数を$K$，中間層のノード数を$J$としたときのニューラルネットワークの式を以下のように定義する。  
$\left[\begin{matrix}g_{1}\\\vdots\\g_{J} \end{matrix} \right] = 
\left[\begin{matrix} w_{11}^{\rm mid}x_1 +\cdots+ w_{1D}^{\rm mid}x_D + b_{1}^{\rm mid}\\ \vdots \\ w_{J1}^{\rm mid}x_1 +\cdots+ w_{JD}^{\rm mid}x_D + b_{J}^{\rm mid} \end{matrix} \right] = 
\left[\begin{matrix} w_{11}^{\rm mid} &\cdots& w_{1D}^{\rm mid} \\  & \vdots& \\ w_{J1}^{\rm mid} &\cdots& w_{JD}^{\rm mid} \end{matrix} \right]\left[\begin{matrix}x_1\\\vdots\\ x_D\end{matrix} \right] + \left[\begin{matrix}b_{1}^{\rm mid}\\\vdots\\b_{J}^{\rm mid} \end{matrix} \right] = {\bf W}^{\rm mid}{\bf x} + {\bf b}^{\rm mid}$  
$\left[\begin{matrix}h_{1}\\\vdots\\h_{J} \end{matrix} \right] = 
\left[\begin{matrix}{\rm sigmoid}(g_1) \\ \vdots \\ {\rm sigmoid}(g_J)\end{matrix} \right]$  
$\left[\begin{matrix}z_{1}\\\vdots\\z_{K} \end{matrix} \right] = 
\left[\begin{matrix} w_{11}h_1 +\cdots+ w_{1J}h_J + b_{1}\\ \vdots \\ w_{K1}h_1 +\cdots+ w_{KJ}h_J + b_{K} \end{matrix} \right] = 
\left[\begin{matrix} w_{11} & \cdots & w_{1J} \\ & \vdots &  \\ w_{K1} & \cdots & w_{KJ} \end{matrix} \right]\left[\begin{matrix}h_1\\\vdots\\h_J \end{matrix} \right] + \left[\begin{matrix}b_{1}\\\vdots\\b_{K} \end{matrix} \right] = {\bf W}{\bf h} + {\bf b}$  
$\left[\begin{matrix}\hat{y}_{1}\\\vdots\\\hat{y}_{K} \end{matrix} \right] = 
\left[\begin{matrix}{\rm softmax}(z_1) \\ \vdots \\ {\rm softmax}(z_K)\end{matrix} \right]$

${\bf W}^{\rm mod}$ および ${\bf b}^{\rm mod}$ はそれぞれ中間層の重み行列とバイアス項で，サイズは $J \times D$，$J$ です。    
${\bf W}$ および ${\bf b}$ は出力層の重み行列とバイアス項で，サイズは $K \times J$，$K$ です。 

上式に従って，1サンプルデータに対してニューラルネットワークを通し，$\bf \hat{y}$ を計算する関数を以下に定義します。  

In [ ]:
def sigmoid(z):
  '''
      シグモイド関数
  '''
  return (1+np.exp(-1*z))**(-1)

def softmax(z):
  '''
      ソフトマックス関数
      z: 要素数=クラス数のベクトル
      y_hat: クラス毎の確率。要素数=クラス数のベクトル
  '''
  return (np.exp(z) / np.sum(np.exp(z)))

def neural_network(x, W_mid, b_mid, W, b):
  '''
      1サンプルデータに対してニューラルネットワークによるクラス分類を行う
      x: 1サンプルデータ。サイズ=Dのベクトル
      W_mid: 中間層の重み行列。サイズ=(JxD)の行列
      b_mid: 中間層のバイアス。サイズ=Jのベクトル
      W: 出力層の重み行列。サイズ=(KxJ)の行列
      b: 出力層のバイアス。サイズ=Kの行列
      ただし，D:次元数，J:中間層のノード数，K:クラス数である。
      y_hat: 出力層の出力。サイズ=Kのベクトル
      h: 中間層の出力。サイズ=Jのベクトル
  '''
  # 中間層を計算
  g = np.dot(W_mid, x) + b_mid
  h = sigmoid(g)

  # 出力層を計算
  z = np.dot(W, h) + b
  y_hat = softmax(z)

  return y_hat, h


10_03_neural_network.ipynbではneural_network関数の`w_mid_1`を$\left[\begin{matrix} w_{11}^{\rm mid} & w_{12}^{\rm mid} \end{matrix}\right]$，`w_mid_2`を$\left[\begin{matrix} w_{21}^{\rm mid} & w_{22}^{\rm mid} \end{matrix}\right]$と二つのベクトルに分けて定義していましたが，今回は  
変数`W_mid` を${\bf W}^{\rm mid}=\left[\begin{matrix} w_{11}^{\rm mid} &\cdots& w_{1D}^{\rm mid} \\  & \vdots& \\ w_{J1}^{\rm mid} &\cdots& w_{JD}^{\rm mid} \end{matrix} \right]$という行列で定義しています。  
またバイアス項に関しても，10_03_neural_network.ipynbでは$b_{1}^{\rm mid}, b_{2}^{\rm mid}$をそれぞれ`b_mid_1`，`b_mid_2`という二つのスカラー値に分けて定義していましたが，今回は`b_mid`を$\left[\begin{matrix} b_{1}^{\rm mid} & b_{2}^{\rm mid} \end{matrix}\right]$というベクトルで定義しています。  
`W_mid`と`b_mid`がそれぞれベクトル → 行列，スカラ → ベクトル，と変わっても，numpy.dot関数で適切に内積処理をしてくれます。   
出力層に関しては変更していません。

クロスエントロピー損失$L_{ce}$を計算する関数および勾配を計算する関数を定義します。  
勾配の計算式は以下のように定義されます。  
出力層の勾配:  
$\frac{\partial L_{ce}}{\partial w_{kj}} = (\hat{y}_k - y_k)h_j$  
$\frac{\partial L_{ce}}{\partial b_{k}} = \hat{y}_k - y_k$  
中間層の勾配:  
$\frac{\partial L_{ce}}{\partial W_{jd}^{\rm mod}} = \left [ \sum_{k=1}^{K}(\hat{y}_k - y_k)w_{kj} \right ] (1-h_j)h_{j}x_{d}$   
$\frac{\partial L_{ce}}{\partial b_{j}^{\rm mod}} = \left [ \sum_{k=1}^{K}(\hat{y}_k - y_k)w_{kj} \right ] (1-h_j)h_j$  
$k$は出力層のノード(=クラス)の番号($1 \le k \le K$)  
$j$は中間層のノードの番号($1 \le j \le J$)  
$d$は入力層のノード(=入力データの次元)の番号($1 \le d \le D)$  
です。

$y_k$は正解ラベルで，$k$が正解クラスの場合は確率1，不正解クラスの場合は0です。  

In [ ]:
def cross_entropy(y_hat, y_onehot):
  '''
      クロスエントロピー損失を計算する。
      y_hat: マルチクラス回帰によって推定された y
      y_onehot: 正解のクラスの確率が1，それ以外の確率を0とするベクトル
                (one-hotベクトル)
  '''
  return -1.0 * np.sum(y_onehot*np.log(y_hat))


def calc_gradient(y_hat, y_onehot, h, W, x):
  '''
      勾配を計算する
      y_hat: 出力層の出力: サイズ=Kのベクトル
      y_onehot: 正解のクラスの確率が1，それ以外の確率を0とするベクトル
                (one-hotベクトル)
      h: 中間層の出力。サイズ=Jのベクトル
      W: 出力層の重み行列。サイズ=(KxJ)の行列
      x: データ: サイズ=Dのベクトル
      ただし，D:次元数，J:中間層のノード数，K:クラス数である。
      grad_W_mid: W_midの勾配：サイズ=(JxD)の行列
      grad_b_mid: b_midの勾配：サイズ=Jのベクトル
      grad_W: Wの勾配: サイズ=(KxJ)の行列
      grad_b: bの勾配: サイズ=Kのベクトル
  '''
  # 次元数D, 中間層のノード数J, クラス数Kを得る。
  D = np.size(x)
  J = np.size(h)
  K = np.size(y_hat)

  # 出力層の勾配を計算
  prev_grad = y_hat - y_onehot
  grad_W = np.dot(np.array([prev_grad]).T, np.array([h]))
  grad_b = prev_grad

  # 中間層の勾配を計算
  prev_grad = np.dot(prev_grad, W)*(1 - h)*h
  grad_W_mid = np.dot(np.array([prev_grad]).T, np.array([x]))
  grad_b_mid = prev_grad

  return grad_W_mid, grad_b_mid, grad_W, grad_b

関数'calc_gradient'について解説します。  
まず出力層の勾配についてです。  
最初の  
`prev_grad = y_hat - y_onehot`  
では，$\hat{y}_k - y_k$を各$k$について計算しています。  
すなわち`prev_grad`はサイズ=$K$のベクトルです。  

次の  
`grad_W = np.dot(np.array([prev_grad]).T, np.array([self.h[layer-1]]))`  
は，以下の処理を1行で書いたことに相当します。  
```
grad_W = np.zeros([K, J])
for k in range(K):
  for j in range(J):
    grad_W[k, j] = prev_grad[k] * h[j]
```
上記は$\frac{\partial L_{ce}}{\partial w_{kj}} = (\hat{y}_k - y_k)h_j$の計算をforループを使って各$k$，$j$に対して行うコードです。  
この処理を，行列形式でまとめて記述とこうなります。  
$\frac{\partial L_{ce}}{\partial {\bf W}} = 
\left[\begin{matrix} \hat{y}_{1} - y_{1} \\ \vdots \\ \hat{y}_{K} - y_{K}\end{matrix} \right]
\left[\begin{matrix} h_{1} & \cdots & h_{J}\end{matrix} \right]
$  
これをpythonで記述すると  
`grad_W = np.dot(np.array([prev_grad]).T, np.array([h]))`となります。  
`np.array([prev_grad])`および`np.array([h])`は，要素数$K$のベクトル`prev_grad`と要素数$J$のベクトル`h`を，それぞれ$(1\times K)$，$(1\times J)$の行列に変えたことを意味します。  
その後，`np.array([prev_grad]).T`とすることで，`prev_grad`を$(K\times 1)$の行列に転置し，さらに$(1\times J)$に変換された`h`と内積を取っています。  

続いて中間層の勾配についてです。  
`prev_grad = np.dot(prev_grad, W)*(1 - h)*h`  
では，中間層のWおよびbの勾配計算式の共通部分である  
$\left [ \sum_{k=1}^{K}(\hat{y}_k - y_k)w_{kj} \right ] (1-h_j)h_{j}$  
を，各$j$について計算しています。（この処理によって，`pref_grad`はサイズ$K$のベクトルからサイズ$J$のベクトルに更新されます。）  
つまり  
```
prev_grad = np.zeros(J)
for j in range(J):
  prev_grad[j] = np.sum((y_hat - y_onehot)*W[:,j])*(1 - h[j])*h[j]
```
に相当します。  
上の実装の`np.sum((y_hat - y_onehot)*W[:,j])`は$\sum_{k=1}^{K}(\hat{y}_k - y_k)w_{kj}$に対応していますが，これは行列形式でまとめて書くと，  
$\left[ \begin{matrix} \sum_{k=1}^{K}(\hat{y}_k - y_k)w_{k1} & \cdots &  \sum_{k=1}^{K}(\hat{y}_k - y_k)w_{kJ} \end{matrix} \right] =
\left[\begin{matrix} \hat{y}_{1} - y_{1} & \cdots & \hat{y}_{K} - y_{K}\end{matrix} \right]
\left[\begin{matrix} w_{11} & \cdots & w_{1J} \\ & \vdots &  \\ w_{K1} & \cdots & w_{KJ} \end{matrix} \right]
$  
となります。  
いま，`prev_grad`には$\left[\begin{matrix} \hat{y}_{1} - y_{1} & \cdots & \hat{y}_{K} - y_{K}\end{matrix} \right]$が格納されていますので，`np.dot(prev_grad, W)`としてやることで，上記の計算が行えます。あとは`(1 - h)*h`との掛け算を追加するだけです。  
このようにして`prev_grad`を更新した後，`grad_W_mid`の更新は`grad_W`の更新と同じ要領で行えるわけです。


各関数が実装できれば，後は`10_03_neural_network.ipynb`とほとんど同じです。  

各パラメータの初期値を決め，学習を実行します。  
まずは，ノード数を2とした場合です。 

In [ ]:
# 中間層のノード数
num_middle_node = 2
# 学習率
lr = 0.2
# エポック数
num_epochs = 30

np.random.seed(0)
# 中間層の初期値
W_mid =  np.random.randn(num_middle_node, num_dimensions)
b_mid = np.zeros(num_middle_node)
# 出力層の初期値
W =  np.random.randn(num_classes, num_middle_node)
b = np.zeros(num_classes)


# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_history = np.array([])

np.random.seed(0)
for epoch in range(num_epochs):
  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples))
  X_tmp = X[(shuffle_index)]
  Y_tmp = Y[(shuffle_index)]
  Y_onehot_tmp = Y_onehot[(shuffle_index)]
  
  loss = 0
  acc = 0
  for n in range(num_samples):
    x = X_tmp[n,:]
    y_onehot = Y_onehot_tmp[n]

    # ニューラルネットワークに与える
    y_hat, h = neural_network(x, W_mid, b_mid, W, b)
  
    # 勾配の計算
    grad_W_mid, grad_b_mid, grad_W, grad_b = calc_gradient(y_hat, y_onehot, h, W, x)

    # 更新
    W -= lr * grad_W
    b -= lr * grad_b
    W_mid -= lr * grad_W_mid
    b_mid -= lr * grad_b_mid

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y_onehot)
  
  # このエポックにおける識別正解率を算出
  for n in range(num_samples):
    x = X_tmp[n,:]
    y = Y_tmp[n]
    # ニューラルネットワークに与える
    y_hat, h = neural_network(x, W_mid, b_mid, W, b)
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc += 1
  
  loss /= num_samples
  acc /= num_samples
  loss_history = np.append(loss_history, loss)
  acc_history = np.append(acc_history, acc*100)
  print('%d-th epoch: cross entropy = %.3f, accuracy = %.3f%%' % (epoch+1, loss, acc*100))

# クロスエントロピーおよび正解率のプロット
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_history)
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.show()

ノード数が2の場合は，100%の正解率は得られませんでした。

識別境界を可視化します。  


In [ ]:
# x1，x2ごとの格子点の数
grid_size = 200

# 格子点：-3.1から3.1まで200個分，等間隔に区切った数列
grid = np.linspace(-3.1, 3.1, grid_size)

# クラス識別結果を色塗りする領域：200x200の図
# 最後の3は，色情報(RGB=赤・緑・青の3次元ごとの色の強さ)
image = np.zeros([grid_size, grid_size, 3])

color_list = [(0, 0, 1), (1, 0, 0)]

for i in range(grid_size):
  for j in range(grid_size):
    # 格子点のデータ
    x = np.array([grid[j], grid[i]])
    # 格子点データを識別する
    y_hat,_ = neural_network(x, W_mid, b_mid, W, b)
    label = np.argmax(y_hat)
    # 識別結果のクラスの色で，色を塗る
    # (グラフは原点が左下だが，画像は左上が原点なので，上下反転させるために-iとしている)
    image[-i, j, :] = color_list[label]

plt.figure(figsize=(7,7))
# 色塗り結果を表示
plt.imshow(image, alpha=0.4, extent=(-3.1, 3.1, -3.1, 3.1))
# データおよび中間層の識別補助線を表示
x1 = np.linspace(-3, 3)
for j in range(num_middle_node):
  x2 = -1.0 * (b_mid[j] + W_mid[j, 0]*x1) / W_mid[j, 1]
  plt.plot(x1, x2, color='k')
for k in range(num_classes):
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
plt.legend()
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

ノード数が2の場合，識別境界を決める補助線の数が2本ということになります。  
これでは，2クラスを100%分離する境界は作成できません。  

ノード数が3の場合を実行してみます。  
(ソースコードは上のコピペです。)

In [ ]:
# 中間層のノード数
num_middle_node = 3
# 学習率
lr = 0.2
# エポック数
num_epochs = 30

np.random.seed(0)
# 中間層の初期値
W_mid =  np.random.randn(num_middle_node, num_dimensions)
b_mid = np.zeros(num_middle_node)
# 出力層の初期値
W =  np.random.randn(num_classes, num_middle_node)
b = np.zeros(num_classes)


# 損失関数の履歴
loss_history = np.array([])
# 正解率の履歴
acc_history = np.array([])

np.random.seed(0)
for epoch in range(num_epochs):
  # データをシャッフルしなおす。
  # (局所解に陥りにくくなるため)
  shuffle_index = np.random.permutation(np.arange(num_samples))
  X_tmp = X[(shuffle_index)]
  Y_tmp = Y[(shuffle_index)]
  Y_onehot_tmp = Y_onehot[(shuffle_index)]
  
  loss = 0
  acc = 0
  for n in range(num_samples):
    x = X_tmp[n,:]
    y_onehot = Y_onehot_tmp[n]

    # ニューラルネットワークに与える
    y_hat, h = neural_network(x, W_mid, b_mid, W, b)
  
    # 勾配の計算
    grad_W_mid, grad_b_mid, grad_W, grad_b = calc_gradient(y_hat, y_onehot, h, W, x)

    # 更新
    W -= lr * grad_W
    b -= lr * grad_b
    W_mid -= lr * grad_W_mid
    b_mid -= lr * grad_b_mid

    # 損失関数の蓄積
    loss += cross_entropy(y_hat, y_onehot)
  
  # このエポックにおける識別正解率を算出
  for n in range(num_samples):
    x = X_tmp[n,:]
    y = Y_tmp[n]
    # ニューラルネットワークに与える
    y_hat, h = neural_network(x, W_mid, b_mid, W, b)
    # 正解率の蓄積
    if np.argmax(y_hat) == y:
      acc += 1
  
  loss /= num_samples
  acc /= num_samples
  loss_history = np.append(loss_history, loss)
  acc_history = np.append(acc_history, acc*100)
  print('%d-th epoch: cross entropy = %.3f, accuracy = %.3f%%' % (epoch+1, loss, acc*100))

# クロスエントロピーおよび正解率のプロット
plt.figure(figsize=(14,5))
plt.subplot(1,2,1)
plt.plot(loss_history)
plt.xlabel('epoch')
plt.ylabel('loss (cross entropy)')
plt.subplot(1,2,2)
plt.plot(acc_history)
plt.xlabel('epoch')
plt.ylabel('accuracy [%]')
plt.show()

今度は100%の正解率が得られました。  
続いて識別境界を可視化します。  

In [ ]:
# x1，x2ごとの格子点の数
grid_size = 200

# 格子点：-3.1から3.1まで200個分，等間隔に区切った数列
grid = np.linspace(-3.1, 3.1, grid_size)

# クラス識別結果を色塗りする領域：200x200の図
# 最後の3は，色情報(RGB=赤・緑・青の3次元ごとの色の強さ)
image = np.zeros([grid_size, grid_size, 3])

color_list = [(0, 0, 1), (1, 0, 0)]

for i in range(grid_size):
  for j in range(grid_size):
    # 格子点のデータ
    x = np.array([grid[j], grid[i]])
    # 格子点データを識別する
    y_hat,_ = neural_network(x, W_mid, b_mid, W, b)
    label = np.argmax(y_hat)
    # 識別結果のクラスの色で，色を塗る
    # (グラフは原点が左下だが，画像は左上が原点なので，上下反転させるために-iとしている)
    image[-i, j, :] = color_list[label]

plt.figure(figsize=(7,7))
# 色塗り結果を表示
plt.imshow(image, alpha=0.4, extent=(-3.1, 3.1, -3.1, 3.1))
# データおよび中間層の識別補助線を表示
x1 = np.linspace(-3, 3)
for j in range(num_middle_node):
  x2 = -1.0 * (b_mid[j] + W_mid[j, 0]*x1) / W_mid[j, 1]
  plt.plot(x1, x2, color='k')
for k in range(num_classes):
  plt.scatter(X[Y==k,0], X[Y==k,1], color=color_list[k], label='class'+str(k))
plt.legend()
plt.xlabel('x1')
plt.ylabel('x2')
plt.xlim([-3, 3])
plt.ylim([-3, 3])
plt.show()

ノード数が3以上であれば，100%分離可能な境界が作成できていることが分かります。

### 実装について補足
for ループを使わずに，難しい表現を使って1行にまとめて書いた理由は，ソースコードを短くしたかったわけでも，格好よく書きたかったわけでもありません。  
pythonは行列計算が得意な反面，C言語と比べて，**forループの処理が遅い**という欠点があるためです。この欠点は，大規模なデータを使って繰り返し学習を行う処理においては致命的になります。  
そのため，いかにベクトル・行列での計算を利用してforループを無くすかが重要になってきます。  
ただ，forループを使わずに書く場合はバグを生みやすくなります。  
ですので，forループを使った場合と比較して，計算結果が同じになることを確認しながら慎重に実装することが鉄則です。（私がこのプログラムを作る場合もそうしています。）